# 1.0 Get the raw news information
* 本篇代码用来获取相关公司的新闻文本
* 由于信息渠道广泛，将会爬虫数个新闻网站: [新浪新闻，微信公众号]

### 1.1 新浪新闻获取

In [ ]:
import re

import pandas as pd
import xlwt as xlwt

print("Hello, news")
from datetime import datetime
now=datetime.now()
import numpy as np
from pylab import mpl
# todo: solve chinese problem for plt
mpl.rcParams['font.sans-serif']=['SimHei']
mpl.rcParams['axes.unicode_minus']=False
import requests
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import json
from pyppeteer import launch
from bs4 import BeautifulSoup
from pyppeteer import launcher
launcher.DEFAULT_ARGS.remove("--enable-automation")
print("Load time ",datetime.now()-now)

In [ ]:
from requests import RequestException


class newsToolForSina:
    @staticmethod
    def get_list(url):

        # 新闻链接
        res=requests.get(url)
        res.encoding='utf-8'

        # 完整HTML
        html=BeautifulSoup(res.text,'html.parser')

        # 新闻列表
        newList=[]

        for item in html.find_all('div',['news-item','img-news-item']):
            try:
                newObj={}
                newObj['title']=item.select('h2 a')[0].text
                newObj['url']=item.select('h2 a')[0].get('href')
                newList.append(newObj)
            except:
                print('出现异常')
        return newList

    @staticmethod
    def get_HtmlDetail_Sina(url):

        # 新闻链接
        res=requests.get(url)
        res.encoding='utf-8'

        # 完整HTML
        html=BeautifulSoup(res.text,'html.parser')

        # 新闻对象
        result={}

        # 新闻标题
        result['title']=html.select('.main-title')[0].text

        # 发布时间
        timesource=html.select('.date-source span')[0].text
        createtime=datetime.strptime(timesource,'%Y年%m月%d日 %H:%M')
        createtime.strftime('%Y-%m-%d')
        result['createtime']=createtime

        # 新闻来源
        result['place']=html.select('.date-source a')[0].text

        # 新闻内容
        article=[]
        for p in html.select('#article p')[:-1]:
            article.append(p.text.strip())
        articleText=' '.join(article)
        result['article']=articleText

        # 新闻作者
        result['author']=html.select('.show_author')[0].text.strip('责任编辑：')

        # 新闻链接
        result['url']=url

        return result

    @staticmethod
    def get_page_news(browser,urlList):
        #获取当前页面所有包含新闻的a标签
        news = browser.find_elements("xpath",'//div[@class="d_list_txt"]/ul/li/span/a')
        if len(news)==0:
            return np.nan
        for i in news:
            link = i.get_attribute('href') #得到新闻url
            # print(len(urlList),link)
            if link not in urlList:  #通过url去重
                urlList.append(link)
        return urlList

    @staticmethod
    def getNewsData(url):
        # 获取新闻的详细信息
        html = newsToolForSina.get_response(url)
        #使用beautifulsoup进行解析
        soup = BeautifulSoup(html,'lxml')

        #标题
        '''
        <h1 class="main-title">证监会要求北京银行说明是否串通*ST康得管理层舞弊</h1>
        '''
        title = soup.select('.main-title')
        #可能有小部分标题的标签不是上述格式 对其进行补充
        if not title:
            title = soup.select('#artibodyTitle')
        if title:
            title = title[0].text
        # print(title)

        #日期
        '''
        <span class="date">2019年07月20日 16:52</span>
        '''
        date = soup.select('.date')
        # 可能有小部分日期的标签不是上述格式 对其进行补充
        if not date:
            date = soup.select('#pub_date')
        if date:
            date = date[0].text
        # print(date)

        #来源
        '''
        <span class="source ent-source">中国证券报</span>
        '''
        source = soup.select('.source')
        # 可能有小部分来源的标签不是上述格式 对其进行补充
        if not source:
            source = soup.select('[data-sudaclick="media_name"]')
        if source:
            source = source[0].text
        # print(source)

        #正文
        article = soup.select('div[class="article"] p')
        # 可能有小部分正文的标签不是上述格式 对其进行补充
        if not article:
            article = soup.select('div[id="artibody"] p')
        if article:
            #把正文放在一个列表中 每个p标签的内容为列表的一项
            article_list = []
            for i in article:
                # print(i.text)
                article_list.append(i.text)
        #转为字典格式
        news = {'link': url, 'title': title, 'date': date, 'source': source, 'article': article_list}
        return news

    @staticmethod
    def get_response(url):
        try:
            #添加User-Agent，放在headers中，伪装成浏览器
            headers = {
                'User-Agent':'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/75.0.3770.100 Safari/537.36'
            }
            response = requests.get(url,headers=headers)
            if response.status_code == 200:
                response.encoding = 'utf-8'
                return response.text
            return None
        except RequestException:
            return None


In [ ]:
###test
# newList=newsToolForSina.get_list('https://news.sina.com.cn/world/')
# resultList=pd.DataFrame()
#
# for i,item in enumerate(newList):
#     try:
#         result=newsToolForSina.get_HtmlDetail_Sina(item['url'])
#         newObj=pd.DataFrame(result,index=[0])
#         resultList=resultList.append(newObj)
#         print (str(i),'写入成功')
#     except BaseException as err:
#         print (str(i),'出现异常: ',err)
#
# resultList.to_csv("newsData/test.csv",header=resultList.columns)
# resultList

## 注意: 新浪新闻的历史数据只能查到近期一个月的，并不能查之前的，所以新浪的可以放弃一部分了

In [ ]:
# from selenium.common import NoSuchElementException
#
# #打开浏览器
# UrlList=[]
# browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# browser.implicitly_wait(10)
# #打开网址
# browser.get('https://news.sina.com.cn/roll/')
# #获取当前页面新闻的url
# UrlList=newsToolForSina.get_page_news(browser,urlList=UrlList)
# while True:
#     try:
#         #找到下一页按钮 并点击
#         '''
#         <a href="javascript:void(0)" onclick="newsList.page.next();return false;">下一页</a>
#         '''
#         browser.find_element("xpath",'//a[@onclick="newsList.page.next();return false;"]').click()
#         #获取下一页新闻的url
#         result=newsToolForSina.get_page_news(browser,urlList=UrlList)
#         if result is np.nan:
#             break
#     except BaseException as err:
#         print(err)
#         browser.close()
#
#
# print("Have found {} URL records, trying to get news text data".format(len(UrlList)))
# resultList=pd.DataFrame()
# for i,item in enumerate(UrlList):
#     try:
#         result=newsToolForSina.getNewsData(item)
#         newObj=pd.DataFrame(result)
#         resultList=resultList.append(newObj)
#         print (str(i),'写入成功')
#         # break
#     except BaseException as err:
#         print (str(i),'出现异常: ',err)
#         # break
#
#
# resultList.to_csv("newsData/SinaNewsRawData.csv",header=resultList.columns)
# resultList

In [ ]:
resultList=pd.read_csv("newsData/SinaNewsRawData.csv")
resultList

### 1.2 新闻联播文本稿

### 1.3 微博基于关键字内容爬取 (主要内容: 文章文本，观看人数，评论，点赞)

### 1.4 微信公众号基于关键字爬取 (主要内容: 文章文本，评论，点赞，收藏)

### 1.5 卫生办 新冠病毒 人数统计(日)

In [ ]:
import asyncio
import nest_asyncio

nest_asyncio.apply()
class newsToolForHealthCenter:
    @staticmethod
    def get_page_news(keywords:[],browser,urlList:{},endTime:str):
        
        news = browser.find_elements("xpath",'/html/body/div[2]/div/div[1]//*[@class="wordGuide Residence-permit"]//*[@class="bigTit clearfix"]/a')
        similarArticles=browser.find_elements("xpath",'/html/body/div[2]/div/div[1]//*[@class="wordGuide Residence-permit"]//*[@class="listInfoCon clearfix "]//*[@class="listIntro wh100 fl"]//*[@class="similarArticles"]//*[@class="similarArticlesList"]/ul//*[@class="clearfix"]/a')
        if len(news)==0:
            return np.nan
        print("--------------new page--------------")
        if len(similarArticles)>0:
            for article in similarArticles:
                # print("-----------  ------------")
                # print(article.get_attribute("text"))
                # print(article.get_attribute("href"))
                similartitle=article.get_attribute("text").strip()
                similarlink=article.get_attribute("href").strip()
                for keyword in keywords:
                    if re.match(pattern=keyword,string=similartitle):
                        print("Found similar article: ",similartitle)
                        if similarlink not in urlList.keys():
                            urlList[similarlink]=similartitle
        for i in news:
            title=i.get_attribute("title").strip()
            link = i.get_attribute('href').strip() #得到新闻url
            for keyword in keywords:
                if re.match(pattern=keyword,string=title):
                    print("Found: ",title)
                    if link not in urlList.keys():
                        urlList[link]=title
        return urlList
    
    @staticmethod
    async def getPageContent(url:str):
        publicDate=""
        source=""
        content=""
        dateRe="发布时间：.?"
        sourceRe="来源:.?"
        # 获取新闻的详细信息
        html=await newsToolForHealthCenter.pyppteer_fetchUrl(url)
        if html is None:
            print(url,"  is nome type")
            return {"Public_Date":np.nan,"Source":np.nan,"Content":np.nan}
        
        elif len(html)<=0:
            return {"Public_Date":np.nan,"Source":np.nan,"Content":np.nan}
        #使用beautifulsoup进行解析
        soup = BeautifulSoup(html,'lxml')

        #日期
        '''
        <source>
            <span>发布时间：2022-09-02</span>
        '''
        origin = soup.find_all('span')
        # 可能有小部分日期的标签不是上述格式 对其进行补充
        for ele in origin:
            if re.match(string=ele.text.strip(),pattern=dateRe):
                # print(ele.text)
                publicDate=ele.text.replace("发布时间：","").strip()
            if re.match(string=ele.text.strip(),pattern=sourceRe):
                # print(ele.text)
                source=ele.text.replace("来源:","").strip()
                
        article=""
        try: 
            #正文
            article = soup.find(id="xw_box").text.strip()
            # print("---------------")
            # print(article)
            content=article
        except BaseException as err:
            return {"Public_Date":np.nan,"Source":np.nan,"Content":np.nan}
        
        return {"Public_Date":publicDate,"Source":source,"Content":content}

    @staticmethod
    async def pyppteer_fetchUrl(url):
        browser = await launch({'headless': False,'dumpio':True, 'autoClose':True})
        page = await browser.newPage()
        await page.goto(url)
        await page.waitFor(100)
        await asyncio.wait([page.waitForNavigation(timeout=500000)])
        str = await page.content()
        await browser.close()
        return str
    
    # @staticmethod
    # async def fetchUrl(url):
    #     return await newsToolForHealthCenter.pyppteer_fetchUrl(url)

In [ ]:
# import asyncio
# import nest_asyncio
# 
# # 👇️ call apply()
# nest_asyncio.apply()
# url = 'http://www.nhc.gov.cn/yjb/s7860/202209/e0a18445e0ab47608527b9c910f77699.shtml'
# 
# async def fetchUrl(url):
#     browser = await launch({'headless': False,'dumpio':True, 'autoClose':True})
#     page = await browser.newPage()
# 
#     await page.goto(url)
#     await asyncio.wait([page.waitForNavigation()])
#     str = await page.content()
#     await browser.close()
#     print("-------------------")
#     print(str)
# 
# asyncio.get_event_loop().run_until_complete(fetchUrl(url))

In [ ]:
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import wait
from selenium.common import NoSuchElementException

#打开浏览器
# browser = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

options = Options()
browser = webdriver.Chrome(service=Service(ChromeDriverManager( latest_release_url='https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json', driver_version='116.0.5845.96').install()), options=options)

browser.implicitly_wait(100)
#打开网址
# browser.get('http://zs.kaipuyun.cn/s?searchWord=%25E6%2596%25B0%25E5%259E%258B%25E5%2586%25A0%25E7%258A%25B6%25E7%2597%2585%25E6%25AF%2592%25E8%2582%25BA%25E7%2582%258E&column=%25E6%259C%25AC%25E7%25AB%2599&pageSize=10&pageNum=0&siteCode=N000001642&sonSiteCode=&checkHandle=1&searchSource=0&areaSearchFlag=0&secondSearchWords=&topical=&docName=&label=&countKey=0&uc=0&left_right_index=0&searchBoxSettingsIndex=&isSecondSearch=undefined&manualWord=%25E6%2596%25B0%25E5%259E%258B%25E5%2586%25A0%25E7%258A%25B6%25E7%2597%2585%25E6%25AF%2592%25E8%2582%25BA%25E7%2582%258E&orderBy=0&startTime=&endTime=&timeStamp=0&strFileType=&wordPlace=0')

browser.get('http://zs.kaipuyun.cn/s?searchWord=%25E6%2596%25B0%25E5%259E%258B%25E5%2586%25A0%25E7%258A%25B6%25E7%2597%2585%25E6%25AF%2592%25E8%2582%25BA%25E7%2582%258E%25E7%2596%25AB%25E6%2583%2585%25E6%259C%2580%25E6%2596%25B0%25E6%2583%2585%25E5%2586%25B5&column=%25E6%259C%25AC%25E7%25AB%2599&pageSize=10&pageNum=0&siteCode=N000001642&sonSiteCode=&checkHandle=1&searchSource=1&areaSearchFlag=0&secondSearchWords=&topical=&docName=&label=&countKey=0&uc=0&left_right_index=0&searchBoxSettingsIndex=&isSecondSearch=undefined&manualWord=%25E6%2596%25B0%25E5%259E%258B%25E5%2586%25A0%25E7%258A%25B6%25E7%2597%2585%25E6%25AF%2592%25E8%2582%25BA%25E7%2582%258E%25E7%2596%25AB%25E6%2583%2585%25E6%259C%2580%25E6%2596%25B0%25E6%2583%2585%25E5%2586%25B5&orderBy=0&startTime=&endTime=&timeStamp=0&strFileType=&wordPlace=0')

#获取当前页面新闻的url
keywords=["截至.*新型冠状病毒肺炎疫情最新情况",'.*新型冠状病毒感染的肺炎疫情情况']
endTime="2018-6-6"
urlList={}
urlList=newsToolForHealthCenter.get_page_news(keywords,browser,urlList=urlList,endTime=endTime)

while True:
    try:
        #找到下一页按钮 并点击
        browser.find_element("xpath",'//*[@id="pageInfo"]//*[@class="next"]/a').click()
        #获取下一页新闻的url
        result=newsToolForHealthCenter.get_page_news(keywords,browser,urlList=urlList,endTime=endTime)
        if result is np.nan:
            break
    except BaseException as err:
        # print(err)
        browser.close()
        break

In [ ]:
print(len(urlList))

In [ ]:
###其中有一个文章链接会报错
allHealthCenterNews=pd.DataFrame()
for link,title in urlList.items():
    contentAndTime=asyncio.run(newsToolForHealthCenter.getPageContent(url=link))
    theNewsInstance={"Title":[title],"Link":[link],"Content":[contentAndTime['Content']],"Public_Time":[contentAndTime['Public_Date']],"Source":[contentAndTime['Source']]}
    theNewsDf=pd.DataFrame.from_dict(theNewsInstance)
    allHealthCenterNews=pd.concat([theNewsDf,allHealthCenterNews])

In [ ]:
allHealthCenterNews

In [ ]:
allHealthCenterNews.to_csv("stockData/furtherInformation/covid19/CovidNews01.csv",index=False,encoding="utf-8")

In [11]:
allHealthCenterNews=pd.read_csv("stockData/furtherInformation/covid19/CovidNews.csv")
allHealthCenterNews

,Title,Link,Content,Public_Time,Source
0,1月23日新型冠状病毒感染的肺炎疫情情况,http://www.nhc.gov.cn/yjb/s3578/202001/5d19a4f...,截至1月22日24时，我委收到国内25个省（区、市）累计报告新型冠状病毒感染的肺炎确诊病例5...,2020-01-23,卫生应急办公室
1,1月22日新型冠状病毒感染的肺炎疫情情况,http://www.nhc.gov.cn/yjb/s3578/202001/a3c8b51...,2020年1月21日0-24时，我委收到国内13省（区、市）报告新增新型冠状病毒感染的肺炎确...,2020-01-22,卫生应急办公室
2,1月21日新型冠状病毒感染的肺炎疫情情况,http://www.nhc.gov.cn/yjb/s3578/202001/930c021...,2020年1月20日0-24时，我委收到国内3省（区、市）报告新增新型冠状病毒感染的肺炎确诊...,2020-01-21,卫生应急办公室
3,1月24日新型冠状病毒感染的肺炎疫情情况,http://www.nhc.gov.cn/yjb/s3578/202001/c5da49c...,1月23日0-24时，27个省（区、市）报告新增确诊病例259例，新增死亡患者8例。新增治愈...,2020-01-24,卫生应急办公室
4,截至11月26日24时新型冠状病毒肺炎疫情最新情况,http://www.nhc.gov.cn/yjb/s7860/202111/322b052...,11月26日0—24时，31个省（自治区、直辖市）和新疆生产建设兵团报告新增确诊病例25例。...,2021-11-27,卫生应急办公室
...,...,...,...,...,...
881,截至9月30日24时新型冠状病毒肺炎疫情最新情况,http://www.nhc.gov.cn/yjb/s7860/202110/0a5882f...,9月30日0—24时，31个省（自治区、直辖市）和新疆生产建设兵团报告新增确诊病例34例。其...,2021-10-01,卫生应急办公室
882,截至10月4日24时新型冠状病毒肺炎疫情最新情况,http://www.nhc.gov.cn/yjb/s7860/202010/9fac34d...,10月4日0—24时，31个省（自治区、直辖市）和新疆生产建设兵团报告新增确诊病例20例，均...,2020-10-05,卫生应急办公室
883,截至10月4日24时新型冠状病毒肺炎疫情最新情况,http://www.nhc.gov.cn/yjb/s7860/202110/cfe33a2...,10月4日0—24时，31个省（自治区、直辖市）和新疆生产建设兵团报告新增确诊病例26例，均...,2021-10-05,卫生应急办公室
884,截至10月6日24时新型冠状病毒肺炎疫情最新情况,http://www.nhc.gov.cn/yjb/s7860/202010/2bf38d3...,10月6日0—24时，31个省（自治区、直辖市）和新疆生产建设兵团报告新增确诊病例7例，均为...,2020-10-07,卫生应急办公室
